In [1]:
import numpy as np
import matplotlib as mpl
import seaborn as sns
sns.set()
%matplotlib inline

import pandas as pd

import json
import glob
import os

In [2]:
paths = glob.glob(os.environ['TDCCH_EXP_OUTPUT_DIR'] + "/query_stats_all_graphs/*.json")
data = [json.load(open(path)) for path in paths]

In [3]:
query_running_times = pd.DataFrame.from_records([{ 
    'graph': run['args'][1],
    'A*': True,
    **algo } 
    for run in data for algo in run["algo_runs"] if algo['algo'] == "Floating TDCCH Query"])

In [4]:
paths = glob.glob(os.environ['TDCCH_EXP_OUTPUT_DIR'] + "/query_stats_no_astar_all_graphs/*.json")
data = [json.load(open(path)) for path in paths]

query_running_times = query_running_times.append(pd.DataFrame.from_records([{ 
    'graph': run['args'][1],
    'A*': False,
    **algo } 
    for run in data for algo in run["algo_runs"] if algo['algo'] == "Floating TDCCH Query"]))

In [34]:
table = query_running_times \
    .groupby(['graph', 'A*']) \
    [['num_nodes_in_elimination_tree_search_space', 'num_relaxed_elimination_tree_arcs', 'num_settled_nodes', 'num_relaxed_shortcut_arcs', 'running_time_ms']] \
    .mean() \
    .unstack()

table['num_nodes_in_elimination_tree_search_space_joined'] = table['num_nodes_in_elimination_tree_search_space'][False].round().astype(int)
table['num_relaxed_elimination_tree_arcs_joined'] = table['num_relaxed_elimination_tree_arcs'][False].round().astype(int)

table['num_settled_nodes'] = table['num_settled_nodes'].round().astype(int)
table['num_relaxed_shortcut_arcs'] = table['num_relaxed_shortcut_arcs'].round().astype(int)

table[['num_nodes_in_elimination_tree_search_space_joined', 'num_relaxed_elimination_tree_arcs_joined', 'num_settled_nodes', 'num_relaxed_shortcut_arcs', 'running_time_ms']] \
    .loc[[os.environ['TDCCH_GRAPH_DIR'] + graph for graph in ['de/day/dido/', 'ptv17-de-car/day/di/', 'ptv17-eur-car/day/di/']]] \
    .rename(index={ os.environ['TDCCH_GRAPH_DIR'] + old: new for (old, new) in [('de/day/dido/', 'Ger06'), ('ptv17-de-car/day/di/', 'Ger17'), ('ptv17-eur-car/day/di/', 'Eur17')] }) \
    .rename(columns={'num_nodes_in_elimination_tree_search_space_joined': '#nodes in CCH search space', 
                     'num_relaxed_elimination_tree_arcs_joined': '#relaxed shortcuts',
                     'num_settled_nodes': '#settled nodes',
                     'num_relaxed_shortcut_arcs': '#evaluated/unpacked arcs',
                     'running_time_ms': 'running time [ms]',}) \
    .to_latex(float_format='{:0.3f}'.format) \
    .replace('False', 'No') \
    .replace('True', 'Yes')

'\\begin{tabular}{lrrrrrrrr}\n\\toprule\n{} & \\#nodes in CCH search space & \\#relaxed shortcuts & \\multicolumn{2}{l}{\\#settled nodes} & \\multicolumn{2}{l}{\\#evaluated/unpacked arcs} & \\multicolumn{2}{l}{running time [ms]} \\\\\nA* &          No &  Yes &                    No &  Yes &             No &  Yes \\\\\ngraph &                            &                    &                &       &                          &       &                   &       \\\\\n\\midrule\nGer06 &                        756 &              52843 &           3410 &   865 &                     3952 &  1042 &             1.646 & 0.563 \\\\\nGer17 &                        790 &              59692 &          18881 &  3097 &                    20401 &  3502 &            12.689 & 1.542 \\\\\nEur17 &                       1430 &             206142 &          43872 &  6915 &                    48917 &  8197 &            25.421 & 3.987 \\\\\n\\bottomrule\n\\end{tabular}\n'